#### Loading packages

In [1]:
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm
from pulp import *
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))


import warnings
warnings.filterwarnings("ignore")

#### Load data

In [2]:
exp_demand = pd.read_excel("Logistics Planning.xlsx", sheet_name="Expected Demand")
exp_demand.set_index("Customer", inplace=True)


shipping_cost = pd.read_excel("Logistics Planning.xlsx", sheet_name="Shipping Costs")
shipping_cost.set_index("Customer", inplace=True)


display_side_by_side(dfs=[exp_demand.head(), 
                          shipping_cost.head()], 
                     captions=["Monthly demand of customers", 
                               "Shipping costs from warehouse locations"])

#### Decision variables

In [3]:
list_of_customers = exp_demand.index.to_list()

list_demand_months = exp_demand.columns.to_list()

list_warehouse_locations = shipping_cost.columns.to_list()

var_list = [(c, m, w) for c in list_of_customers for m in list_demand_months for w in list_warehouse_locations]

var_dict = LpVariable.dicts(name = "Shipment", 
                            indexs = var_list, 
                            lowBound = 0, 
                            cat = "Integer")

print("Number of decision variables = ", len(var_list))

Number of decision variables =  4824


#### Model initialization

In [4]:
model  = LpProblem("Minimize Shipping Costs", LpMinimize)

#### Define objective function

In [5]:
model += lpSum([shipping_cost.loc[c,w]*var_dict[(c, m, w)] 
                for c in list_of_customers for w in list_warehouse_locations for m in list_demand_months])

####  Define demand constraints

In [6]:
for m in list_demand_months:
    for c in list_of_customers:
        model += lpSum([var_dict[(c,m,w)] for w in list_warehouse_locations]) == exp_demand.loc[c, m]

#### Solve the model

In [7]:
model.solve()

if LpStatus[model.status]=="Optimal":
    optimal_solution = pd.DataFrame([(v.name, v.varValue) for v in model.variables() if v.varValue!=0], 
                                    columns=["Decision_Variables", "Number of Shipments"])

display_side_by_side(dfs = [optimal_solution[optimal_solution["Decision_Variables"].str.contains(m)].head()\
                            for m in list_demand_months[:6]], 
                     captions = ["Plan for {}".format(m) for m in list_demand_months[:6]])

,Decision_Variables,Number of Shipments
4,"Shipment_('Customer_1',_'January',_'New_York')",8196.000000
16,"Shipment_('Customer_10',_'January',_'San_Jose')",4979.000000
28,"Shipment_('Customer_11',_'January',_'Chicago')",3335.000000
40,"Shipment_('Customer_12',_'January',_'San_Jose')",7339.000000
52,"Shipment_('Customer_13',_'January',_'Boston')",8551.000000
,Decision_Variables,Number of Shipments
3,"Shipment_('Customer_1',_'February',_'New_York')",5138.000000
15,"Shipment_('Customer_10',_'February',_'San_Jose')",3188.000000
27,"Shipment_('Customer_11',_'February',_'Chicago')",6957.000000
39,"Shipment_('Customer_12',_'February',_'San_Jose')",3393.000000
